In [1]:
import json 
import pandas as pd 

In [2]:
filepath = './spiders/pdf_urls.json'

with open(filepath) as fp:
    data = json.load(fp)
df = pd.DataFrame(data)
df.head()

,Orgname,Org_pdf_url
0,Tinna Trade Ltd.,https://trendlyne.com/get-document/post/pdf/47...
1,PPAP Automotive Ltd.,https://trendlyne.com/get-document/post/pdf/47...
2,Subros Ltd.,https://trendlyne.com/get-document/post/pdf/47...
3,SKF India Ltd.,https://trendlyne.com/get-document/post/pdf/47...
4,Hindustan Oil Exploration Company Ltd.,https://trendlyne.com/get-document/post/pdf/47...


In [3]:
df.isna().sum()

Orgname           0
Org_pdf_url    2655
dtype: int64

In [4]:
df.describe()

,Orgname,Org_pdf_url
count,25909,23254
unique,1219,18670
top,Multi Commodity Exchange of India Ltd.,None
freq,107,3


In [5]:
df.drop_duplicates(subset=['Org_pdf_url'], keep='first', inplace=True)
df.dropna(subset=['Org_pdf_url'], inplace=True)

In [6]:
df.describe()

,Orgname,Org_pdf_url
count,18670,18670
unique,1197,18670
top,Multi Commodity Exchange of India Ltd.,https://trendlyne.com/get-document/post/pdf/47...
freq,92,1


In [7]:
df.to_json('data.json', orient='records', lines=True)
json_str = ',\n'.join(json.dumps(row) for row in df.to_dict(orient='records'))
with open('data.json', 'w') as f:
    f.write('[' + json_str + ']')
    


In [ ]:
import os
import json
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

dump_folder = 'pdf_dump'
os.makedirs(dump_folder, exist_ok=True)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

session = requests.Session()
session.headers.update(headers)

def login(session, login_url, email, password):
    print(f"Attempting login at {login_url}")
    response = session.get(login_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # You may need to adjust these selectors based on the actual login form
    login_form = soup.find('form')
    if not login_form:
        print("Login form not found")
        return False
    
    login_action = login_form.get('action')
    if login_action:
        login_url = urljoin(login_url, login_action)
    
    login_data = {
        'email': email,
        'password': password,
    }
    
    # Add any additional hidden fields from the form
    for input_field in login_form.find_all('input', type='hidden'):
        login_data[input_field.get('name')] = input_field.get('value')
    
    response = session.post(login_url, data=login_data)
    
    # Check if login was successful (you may need to adjust this condition)
    if "Login Successful" in response.text or response.url != login_url:
        print("Login successful")
        return True
    else:
        print("Login failed")
        return False

def download_pdf(url, folder, filename):
    try:
        response = session.get(url, allow_redirects=False)
        
        if response.status_code == 302 and 'login' in response.headers.get('Location', '').lower():
            login_url = response.headers['Location']
            print(f"Redirected to login page: {login_url}")
            
            # You need to provide your actual login credentials here
            email = 'icarusvp@gmail.com'
            password = '15TEEN$ollar'
            
            if login(session, login_url, email, password):
                # Retry the download after successful login
                return download_pdf(url, folder, filename)
            else:
                print("Login failed, cannot download PDF")
                return False
        
        response.raise_for_status()
        
        if response.headers.get('Content-Type') == 'application/pdf':
            file_path = os.path.join(folder, filename)
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {filename}")
            return True
        else:
            print(f"Failed to download {filename}: Content is not a PDF")
            print(response.text)
            return False
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {filename}: {e}")
        return False

# Control the number of downloads
max_downloads = 1  # Set the maximum number of downloads you want
download_count = 0

# Iterate through the JSON data and download each PDF
for entry in data:
    if download_count >= max_downloads:
        break
    
    org_name = entry["Orgname"]
    pdf_url = entry["Org_pdf_url"]
    pdf_filename = f"{org_name}.pdf".replace(" ", "_")  # Sanitize filename
    
    if download_pdf(pdf_url, dump_folder, pdf_filename):
        download_count += 1

print(f"Downloaded {download_count} PDFs.")